### 10-01 RAG 실습

In [17]:
import os
import psycopg2
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

# OpenAI 클라이언트
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Postgres 접속정보
db_config = {
    "host": os.getenv("DB_HOST"),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASS"),
    "port": os.getenv("DB_PORT"),
}

# 임베딩 생성
def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
    resp = client.embeddings.create(input=text, model=model)
    return resp.data[0].embedding

# 가장 유사한 문장 검색
def find_most_similar(text_vector: list) -> str:
    # pgvector 리터럴: [v1,v2,...] 형태
    vector_str = "[" + ",".join(map(str, text_vector)) + "]"

    sql = """
        SELECT name, embedding <-> %s::vector AS distance
        FROM test_vector
        ORDER BY distance ASC
        LIMIT 3;
    """

    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()
    try:
        cur.execute(sql, (vector_str,))
        rows = cur.fetchall()
        return "\n".join(row[0] for row in rows)
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        return ""
    finally:
        cur.close()
        conn.close()

# ChatGPT 호출
def ask_chatgpt(context: str, question: str, model: str = "gpt-3.5-turbo") -> str:
    prompt = f"""
다음 문장을 참고하여 질문에 답해주세요.

[참고 문장]
{context}

[질문]
{question}
""".strip()

    print("prompt(START)------------------------------------------------------------\n")
    print(prompt)
    print("prompt(END)--------------------------------------------------------------\n")

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 친절한 지식 도우미입니다."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
    )
    return resp.choices[0].message.content.strip()

if __name__ == "__main__":
    user_question = input("질문을 입력하세요: ").strip()
    # 사용자 질문 임베딩
    question_vector = get_embedding(user_question)
    # 가장 유사한 문장 검색
    context_text = find_most_similar(question_vector)

    if context_text:
        answer = ask_chatgpt(context_text, user_question)
        print(f"\nchatgpt의 답변:\n{answer}")
    else:
        print("X 유사한 문장을 찾을 수 없습니다")

prompt(START)------------------------------------------------------------

다음 문장을 참고하여 질문에 답해주세요.

[참고 문장]
할머니의 생신은 1955년 2월 30일 입니다.
남산위에 저소나무 철갑을 두른듯
남산위에 저 소나무 철갑을 두른듯 바람서리 불변함은 우리 기상일세

[질문]
할머니 생신은?
prompt(END)--------------------------------------------------------------


chatgpt의 답변:
문장에서는 할머니의 생신이 1955년 2월 30일이라고 나와 있지만, 실제로는 2월에는 28일까지만 있기 때문에 2월 30일은 존재하지 않는 날짜입니다. 따라서, 정확한 생신일을 알 수 없습니다.


In [5]:
import pandas as pd

df = pd.read_csv("data/국민건강보험공단_진료내역정보_2024.CSV", encoding="cp949")
df.head(10)

,기준년도,가입자일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수
0,2024,180755,7014,2,2,41,2024-03-18,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
1,2024,180755,7021,2,2,41,2024-03-14,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
2,2024,180755,7024,2,2,41,2024-03-25,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
3,2024,180755,7028,2,2,41,2024-03-21,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
4,2024,868682,10091,2,2,41,2024-03-15,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
5,2024,831540,30221,2,3,41,2024-03-27,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
6,2024,4160,62893,2,2,41,2024-03-18,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
7,2024,190636,66219,2,2,41,2024-03-18,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
8,2024,253678,69753,2,12,41,2024-03-28,3,1,J209,J304,1,1,0.0,12590,3700,8890,3
9,2024,994921,312129,2,8,41,2024-02-26,3,1,J209,J304,1,1,0.0,12590,3700,8890,3


In [14]:
df.loc[1]

기준년도                2024
가입자일련번호           180755
진료내역일련번호            7021
성별코드                   2
연령대코드                  2
시도코드                  41
요양개시일자        2024-03-14
서식코드                   3
진료과목코드                 1
주상병코드               J209
부상병코드               J304
요양일수                   1
입내원일수                  1
심결가산율                0.0
심결요양급여비용총액         12590
심결본인부담금             3700
심결보험자부담금            8890
총처방일수                  3
Name: 1, dtype: object

In [21]:
a,b,c, *d = [1,2,2,3,4,12,312,3,21,3,123,21,3,13] # d 는 unpack 하고 나머지 값이 리스트화됨
print(d)

[3, 4, 12, 312, 3, 21, 3, 123, 21, 3, 13]


In [ ]:
import numpy as np
vec = np.array([[10,20,30],[40,50,60]])
vec[0]
vec[0].ndim
vec.ndim

2

In [28]:
np.zeros((2,3))
np.zeros_like(vec)

array([[0, 0, 0],
       [0, 0, 0]])

In [3]:
import numpy as np
np.arange(30).reshape((5,6))

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]])

In [16]:
import pandas as pd
pp = pd.read_excel("data/USA_healthcare_dataset.xlsx")
pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                55500 non-null  object        
 1   Age                 55500 non-null  int64         
 2   Gender              55500 non-null  object        
 3   Blood Type          55500 non-null  object        
 4   Medical Condition   55500 non-null  object        
 5   Date of Admission   55500 non-null  datetime64[ns]
 6   Doctor              55500 non-null  object        
 7   Hospital            55500 non-null  object        
 8   Insurance Provider  55500 non-null  object        
 9   Billing Amount      55500 non-null  float64       
 10  Room Number         55500 non-null  int64         
 11  Admission Type      55500 non-null  object        
 12  Discharge Date      55500 non-null  datetime64[ns]
 13  Medication          55500 non-null  object    

In [ ]:
import os
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from dotenv import load_dotenv

load_dotenv()

db_config = {
    'host': os.getenv("DB_HOST"),
    'port': 5432,
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
}


conn = None
cursor = None

try:
    # DB 연결
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # 엑셀 로드 & 컬럼명 정리
    df = pd.read_excel("data/USA_healthcare_dataset.xlsx")
    df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

    # 타입 변환: DB에 맞게 맞춤
    df["date_of_admission"] = pd.to_datetime(df["date_of_admission"]).dt.date   # date
    df["discharge_date"]    = pd.to_datetime(df["discharge_date"]).dt.date      # date
    df["room_number"]       = df["room_number"].astype(str)                     # varchar
    df["age"]               = df["age"].astype(int)                             # int4
    df["billing_amount"]    = df["billing_amount"].astype(float)                # float8

    insert_sql = """
        INSERT INTO usa_health_data (
            name, age, gender, blood_type, medical_condition,
            date_of_admission, doctor, hospital, insurance_provider,
            billing_amount, room_number, admission_type,
            discharge_date, medication, test_results
        )
        VALUES (
            %s, %s, %s, %s, %s,
            %s, %s, %s, %s,
            %s, %s, %s,
            %s, %s, %s
        )
        ON CONFLICT (name, date_of_admission, doctor) DO NOTHING
        RETURNING id;
    """

    # 튜플 데이터 준비
    rows_to_insert = [
        (
            r["name"], r["age"], r["gender"], r["blood_type"], r["medical_condition"],
            r["date_of_admission"], r["doctor"], r["hospital"], r["insurance_provider"],
            r["billing_amount"], r["room_number"], r["admission_type"],
            r["discharge_date"], r["medication"], r["test_results"]
        )
        for _, r in df.iterrows()
    ]

    # 대량 insert
    execute_batch(cursor, insert_sql, rows_to_insert, page_size=2000)
    conn.commit()

    # 최근 10개 데이터 조회
    cursor.execute("""
        SELECT id, name, age, gender, medical_condition
        FROM usa_health_data
        ORDER BY id DESC
        LIMIT 10;
    """)
    recent = cursor.fetchall()

    print("최근 10개 데이터")
    for row in recent:
        print(row)

except Exception as e:
    print("에러 발생:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("DB 연결이 종료되었습니다.")

최근 10개 데이터
(50000, 'kaTheRIne WeBSTer', 23, 'Male', 'Arthritis')
(49999, 'RoGER farRELl', 81, 'Male', 'Cancer')
(49998, 'LiSa sIMPsoN', 73, 'Male', 'Cancer')
(49997, 'braDleY dAniEL', 69, 'Male', 'Cancer')
(49996, 'JOSepH PaUl', 64, 'Male', 'Hypertension')
(49995, 'CaRl CHaNDlEr', 33, 'Female', 'Cancer')
(49994, 'JAMes lITTLe', 25, 'Male', 'Asthma')
(49993, 'REbECcA toRReS', 73, 'Male', 'Arthritis')
(49992, 'sara TAYloR', 63, 'Female', 'Cancer')
(49991, 'tamAra MCcOrmIck', 84, 'Female', 'Arthritis')
DB 연결이 종료되었습니다.


In [26]:
#Tokenizer

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# nltk: 영어 corpus tokenization용 도구
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

print('단어 토큰화1 :', word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))
print('단어 토큰화2 :', WordPunctTokenizer().tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))
print('단어 토큰화3 :', text_to_word_sequence("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hyunjunson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hyunjunson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


단어 토큰화1 : ['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']
단어 토큰화2 : ['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']
단어 토큰화3 : ["don't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', 'mr', "jone's", 'orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']


In [27]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
text = "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."
print('트리뱅크 워드토크나이저 :',tokenizer.tokenize(text))

트리뱅크 워드토크나이저 : ['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


In [ ]:
# 문장단위로 토크나이저 하는 라이브러리

from nltk.tokenize import sent_tokenize
text = """His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the
edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to make sure no one was near."""
print('문장 토큰화1 :',sent_tokenize(text))
text = "I am actively looking for Ph.D. students. and you are a Ph.D student."
print('문장 토큰화2 :',sent_tokenize(text))

문장 토큰화1 : ['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the\nedge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to make sure no one was near.']
문장 토큰화2 : ['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']


In [ ]:
# 한국어 문장단위로 토큰화
import kss
text = "딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?"
print('한국어 문장 토큰화 :',kss.split_sentences(text))

[Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D



한국어 문장 토큰화 : ['딥 러닝 자연어 처리가 재미있기는 합니다.', '그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다.', '이제 해보면 알걸요?']
